<a href="https://colab.research.google.com/github/Matte920/DeepLearning_PracticaFinal/blob/main/Hito4_EarlyFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers


df_meta = pd.read_csv("/content/HAM10000_metadata.csv")
df_img  = pd.read_csv("/content/hnmist_28_28_RGB.csv")


y_str = df_meta["dx"].values
le = LabelEncoder()
y_int = le.fit_transform(y_str)
num_classes = len(le.classes_)
y_oh = keras.utils.to_categorical(y_int, num_classes=num_classes)

X_tab = df_meta[["sex", "age", "localization"]].copy()

X_tab["sex"] = X_tab["sex"].fillna("unknown")
X_tab["localization"] = X_tab["localization"].fillna("unknown")
X_tab["age"] = X_tab["age"].fillna(X_tab["age"].median())

X_tab = pd.get_dummies(X_tab, columns=["sex", "localization"])
X_tab = X_tab.astype("float32")
input_dim_tab = X_tab.shape[1]


X_img = df_img.values.astype("float32")
X_img = X_img / 255.0

num_pixels = X_img.shape[1]
assert num_pixels == 28 * 28 * 3, "Número de píxeles inesperado."

X_img = X_img.reshape(-1, 28, 28, 3)
input_shape_img = (28, 28, 3)


idx = np.arange(len(y_int))

idx_train, idx_test = train_test_split(
    idx,
    test_size=0.2,
    random_state=42,
    stratify=y_int
)

X_train_tab = X_tab.values[idx_train]
X_test_tab  = X_tab.values[idx_test]

X_train_img = X_img[idx_train]
X_test_img  = X_img[idx_test]

y_train = y_oh[idx_train]
y_test  = y_oh[idx_test]


scaler = StandardScaler()
X_train_tab[:, 0:1] = scaler.fit_transform(X_train_tab[:, 0:1])
X_test_tab[:, 0:1] = scaler.transform(X_test_tab[:, 0:1])


tab_input = keras.Input(shape=(input_dim_tab,), name="tabular_input")
t = layers.Dense(64, activation="relu")(tab_input)
t = layers.Dropout(0.3)(t)
t = layers.Dense(32, activation="relu")(t)
feat_tab = layers.Dropout(0.3, name="feat_tab")(t)

img_input = keras.Input(shape=input_shape_img, name="imagen_input")
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(img_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
feat_img = layers.Dropout(0.5, name="feat_img")(x)


fusion = layers.Concatenate(name="fusion_features")([feat_tab, feat_img])

f = layers.Dense(64, activation="relu")(fusion)
f = layers.Dropout(0.5)(f)
output = layers.Dense(num_classes, activation="softmax", name="fusion_output")(f)

model_early = keras.Model(
    inputs=[tab_input, img_input],
    outputs=output,
    name="modelo_early_fusion"
)

model_early.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_early.summary()

history_early = model_early.fit(
    [X_train_tab, X_train_img],
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)


test_loss_early, test_acc_early = model_early.evaluate(
    [X_test_tab, X_test_img],
    y_test,
    verbose=0
)
print("Test accuracy (estrategia EARLY-FUSION – HITO 4):", test_acc_early)

Model: "modelo_early_fusion"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ imagen_input        │ (None, 28, 28, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 28, 28,    │        896 │ imagen_input[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 14, 14,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabular_input       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 14, 14,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,280 │ tabular_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 7, 7, 64)  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3136)      │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │    401,536 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feat_tab (Dropout)  │ (None, 32)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feat_img (Dropout)  │ (None, 128)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_features     │ (None, 160)       │          0 │ feat_tab[0][0],   │
│ (Concatenate)       │                   │            │ feat_img[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     10,304 │ fusion_features[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_output       │ (None, 7)         │        455 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 435,047 (1.66 MB)

 Trainable params: 435,047 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.5430 - loss: 1.4084 - val_accuracy: 0.6843 - val_loss: 0.8809
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.6822 - loss: 0.9721 - val_accuracy: 0.7180 - val_loss: 0.8225
Epoch 3/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.7028 - loss: 0.8799 - val_accuracy: 0.7274 - val_loss: 0.7695
Epoch 4/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.7001 - loss: 0.8665 - val_accuracy: 0.7230 - val_loss: 0.7481
Epoch 5/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.7197 - loss: 0.8226 - val_accuracy: 0.7293 - val_loss: 0.7442
Epoch 6/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.7239 - loss: 0.7875 - val_accuracy: 0.7367 - val_loss: 0.7327
Epoch 7/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.7283 - loss: 0.7487 - val_accuracy: 0.7318 - val_loss: 0.7617
Epoch 8/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.7199 - loss: 0.7683 - val_a

El modelo de early-fusion, que combina las características tabulares e
imágenes en una única red, obtiene una accuracy ≈0.74 en test, similar
o ligeramente mejor que el modelo solo de imágenes.